# 401 Crunch additional NNCE for peak emissions

In this notebook, we assess the additional net-negative emissions to ensure peak warming stays below a desired value.

In [1]:
PEAK = 1.6
ENSEMBLE_MEMBER = 2

In [49]:
import pyam
import scmdata
import copy
import pandas as pd
import json
import os

import dotenv
from climate_assessment.climate.wg3 import clean_wg3_scenarios
import openscm_runner
from utils import assign_peak_and_2100_warming

import matplotlib.pyplot as plt

from pathlib import Path

## Read in the necessary data
Step 1: Read in the prepared metrics.

In [3]:
metrics = pd.read_csv(
    Path(
        '../data/304_compiled_metrics.csv'
    ),
    index_col=[0,1,2]
)

In [4]:
metrics_ensemble = metrics.loc[
    pd.IndexSlice[:,:,ENSEMBLE_MEMBER],
    :
]

Step 2: Read in the emissions dataframe that we will tamper with.

In [5]:
emissions = pyam.IamDataFrame(
    Path(
        '../data/100_scenarios.csv'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/100_scenarios.csv


Step 3: Read in the MAGICC runs for a given ensemble members

In [6]:
dotenv.load_dotenv()

True

In [7]:
os.environ['DYLD_LIBRARY_PATH'] = '/usr/local/Cellar/gcc/13.1.0/lib/gcc/current/'

In [8]:
with open(os.environ["MAGICC_AR6_PROBABILISTIC_DISTRIBUTION"]) as f:
    prob_dist = json.load(f)

display(prob_dist["description"])
cfg_to_run = [
    d for d in prob_dist["configurations"] if d["paraset_id"] == ENSEMBLE_MEMBER
]
assert len(cfg_to_run) == 1
cfg_to_run = {k.lower(): v for k, v in cfg_to_run[0]["nml_allcfgs"].items()}

'IPCC AR6 config drawn on 9th Feb 2021. Sub-sample set 0fd0f62 from derived metrics set f023edb. Drawn in 532_plot_and_save_subsampled_distribution.ipynb.'

## Data processing
Step 1: Select necessary columns from the metrics dataframe.

In [9]:
columns = [
    'peak_warming',
    'year_peak_warming',
    'cum_emissions_to_peak_GtCO2',
    'eTCREup'
]

In [10]:
metrics_small = metrics_ensemble.loc[:,columns]

Step 2: Add a column with the necessary "cooling" to be achieved.

In [11]:
metrics_small.loc[:,'cooling_needs'] = (
    metrics_small
    .apply(
        lambda x: x['peak_warming'] - PEAK if x['peak_warming'] > PEAK else 0,
        axis=1
    )
)

Step 3: Calculate the additional negative emissions needed to achieve this cooling.

In [12]:
metrics_small.loc[:, 'nnce_guess_peak'] = (
    metrics_small.loc[:,'cooling_needs']
    /
    metrics_small.loc[:,'eTCREup']
)

In [13]:
metrics_small

,,,peak_warming,year_peak_warming,cum_emissions_to_peak_GtCO2,eTCREup,cooling_needs,nnce_guess_peak
model,scenario,run_id,,,,,,
COFFEE 1.1,EN_NPi2020_400f_lowBECCS,2,2.108080,2060,983.807875,0.000866,0.508080,586.729568
REMIND-MAgPIE 2.1-4.2,SusDev_SDP-PkBudg1000,2,1.782709,2038,689.232522,0.000764,0.182709,239.117122
REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_ HighRE_Budg900,2,1.821199,2048,698.951810,0.000809,0.221199,273.496172


Step 4: We need a helper function to construct a "DAC" pathway.

In [14]:
def construct_a_cdr_pathway(
    index,
    values
):
    """
    Helper function to construct the pathway
    assuming we have an iterator
    """
    structure = pd.DataFrame(
        {
            'model':index[0],
            'scenario':index[1],
            'variable':'CDR',
            'region':'World',
            'unit':'Gt CO2/yr',
        },
        index=[0]
    )
    # First we set it to 0 DAC between 2015 and 2025
    for t in range(2015, 2025):
        structure.loc[:,t]=0
    # Second, we scale up CDR from 2025 on a yearly basis
    yearly_value = (
        values['nnce_guess_peak']
        /
        (values['year_peak_warming'] - 2025)
    )
    for t in range(2025, int(values['year_peak_warming'])+1):
        structure.loc[:,t]=yearly_value
    # Finally, we set the rest of the values to zero again
    for t in range(int(values['year_peak_warming']+1), 2101):
        structure.loc[:,t]=0
    # Cast to a pyam dataframe and write this out
    return pyam.IamDataFrame(structure)

Step 5: Run through each model-scenario-run_id combination to get the new CDR pathways

In [33]:
def construct_new_pathways(metric_data_input):
    # Step 5.1: Construct the CDR pathways
    dfs = []
    for index, value in metric_data_input.iterrows():
        dfs.append(
            construct_a_cdr_pathway(
                index,
                value
            )
        )
    compiled_cdr_pathways = pyam.concat(
        dfs
    )
    # Step 5.2: COnvert units
    compiled_cdr_pathways.convert_unit(
        current='Gt CO2/yr',
        to='Mt CO2/yr',
        inplace=True
    )
    # Step 5.3: Construct a new CO2 pathway
    co2_pathways_initial = (
        emissions
        .filter(
            variable='AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes'
        )
        .swap_time_for_year()
    )
    # Step 5.4: Append the CDR pathways to this dataframe
    co2_pathways_initial = pyam.concat(
        [
            co2_pathways_initial,
            compiled_cdr_pathways
        ]
    )
    # Step 5.5: Construct a "new" CO2 pathway by subtracting the CDR from the CO2 emissions
    co2_pathways_new = (
        co2_pathways_initial
        .subtract(
            a='AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
            b='CDR',
            name='AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
            ignore_units='Mt CO2/yr'
        )
    )
    # Step 5.6: Compile a new set of emissions to run through MAGICC
    emissions_input = copy.copy(emissions)
    emissions_input.swap_time_for_year(
        inplace=True
    )
    emissions_input.filter(
        variable='AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
        keep=False,
        inplace=True
    )
    emissions_input = pyam.concat(
        [
            emissions_input,
            co2_pathways_new
        ]
    )
    # Step 5.7: Prepare an scmdataframe
    input_scm = scmdata.ScmRun(
        clean_wg3_scenarios(
            emissions_input
        )
    )
    return input_scm

Step 6: Add a helper function to compile metrics based on the output.

In [ ]:
def make_new_metric_df(input_data):
    # Step 6.1: Run MAGICC
    temp_initial_guess = openscm_runner.run(
        {'MAGICC7':[cfg_to_run]},
        input_scm,
        output_variables=[
            'Surface Temperature',
        ]    
    )
    # Step 6.2: Construct metric dataframe
    metrics = (
        assign_peak_and_2100_warming(
            temp_initial_guess
            .relative_to_ref_period_mean(
                year=[1850,1900]
            )
        )
    ) 


In [36]:
temp_initial_guess = openscm_runner.run(
    {'MAGICC7':[cfg_to_run]},
    input_scm,
    output_variables=[
        'Surface Temperature',
    ]
)

Climate models:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

openscm_runner.adapters.magicc7.magicc7 - WARNING: Historical data has not been checked


Writing SCEN7 files:   0%|          | 0.00/3.00 [00:00<?, ?it/s]

openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Entered _parallel_magicc_compact_out
openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Running in parallel with up to 4 workers


Front serial:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

openscm_runner.adapters.magicc7._magicc_instances - INFO: Creating new magicc instance: (7, 'MainProcess') - /Users/gauravganti/Documents/github_projects/cdr_climate_uncertainty/pymagicc-irp9tmpc
openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Setting up MAGICC worker in /Users/gauravganti/Documents/github_projects/cdr_climate_uncertainty/pymagicc-irp9tmpc
openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Writing Pymagicc compatible MAGCFG_USER.CFG in /Users/gauravganti/Documents/github_projects/cdr_climate_uncertainty/pymagicc-irp9tmpc/run
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:436: UserWarning: magicc logged a ERROR message. Check the 'stderr' key of the result's `metadata` attribute.
  warnings.warn(
openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: magicc run stderr: [Aug 18 2023 13:37:13][write_single_variable] <ERROR> Unknown variable DAT_INVERSEEMIS

openscm_runner.adapters.ma

Front parallel:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:436: UserWarning: magicc logged a ERROR message. Check the 'stderr' key of the result's `metadata` attribute.
  warnings.warn(


Parallel runs: 0.00it [00:00, ?it/s]

openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Appending results into a single ScmRun
openscm_runner.adapters.magicc7._magicc_instances - INFO: removing /Users/gauravganti/Documents/github_projects/cdr_climate_uncertainty/pymagicc-irp9tmpc
openscm_runner.adapters.magicc7._magicc_instances - INFO: removing /Users/gauravganti/Documents/github_projects/cdr_climate_uncertainty/pymagicc-rnlbb18n
openscm_runner.adapters.magicc7._run_magicc_parallel - INFO: Shutting down parallel pool
openscm_runner.run - INFO: Only one model run, returning its results


In [27]:
temp_world = temp_initial_guess.filter(
    region='World'
)

In [28]:
temp_world_rebased = (
    temp_world
    .relative_to_ref_period_mean(
        year=[1850,1900]
    )
)

In [32]:
temp_world_rebased.timeseries().max(axis=1)

climate_model  model                  reference_period_end_year  reference_period_start_year  region  run_id  scenario                       unit  variable                      
MAGICCv7.5.3   REMIND-MAgPIE 2.1-4.3  1900                       1850                         World   0       DeepElec_SSP2_ HighRE_Budg900  K     Surface Air Temperature Change    1.710209
               COFFEE 1.1             1900                       1850                         World   1       EN_NPi2020_400f_lowBECCS       K     Surface Air Temperature Change    1.834458
               REMIND-MAgPIE 2.1-4.2  1900                       1850                         World   2       SusDev_SDP-PkBudg1000          K     Surface Air Temperature Change    1.665385
dtype: float64

In [30]:
metrics_small

,,,peak_warming,year_peak_warming,cum_emissions_to_peak_GtCO2,eTCREup,cooling_needs,nnce_guess_peak
model,scenario,run_id,,,,,,
COFFEE 1.1,EN_NPi2020_400f_lowBECCS,2,2.108080,2060,983.807875,0.000866,0.508080,586.729568
REMIND-MAgPIE 2.1-4.2,SusDev_SDP-PkBudg1000,2,1.782709,2038,689.232522,0.000764,0.182709,239.117122
REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_ HighRE_Budg900,2,1.821199,2048,698.951810,0.000809,0.221199,273.496172
